In [1]:
import random
import networkx as nx
from collections import defaultdict

def generate_random_dna(length):
    return ''.join(random.choice('ACGT') for _ in range(length))

def simulate_reads(sequence, read_length, num_reads):
    reads = []
    for _ in range(num_reads):
        start = random.randint(0, len(sequence) - read_length)
        reads.append(sequence[start:start+read_length])
    return reads

def create_kmers(reads, k):
    kmers = []
    for read in reads:
        kmers.extend([read[i:i+k] for i in range(len(read)-k+1)])
    return kmers

def build_de_bruijn_graph(kmers):
    graph = defaultdict(list)
    for kmer in kmers:
        prefix, suffix = kmer[:-1], kmer[1:]
        graph[prefix].append(suffix)
    return dict(graph)

def find_eulerian_path(graph):
    G = nx.DiGraph(graph)
    
    in_degrees = dict(G.in_degree())
    out_degrees = dict(G.out_degree())
    
    start_node = None
    end_node = None
    
    for node in G.nodes():
        if out_degrees[node] - in_degrees[node] == 1:
            if start_node:
                print("Multiple start nodes found. Graph is not Eulerian.")
                return None, G
            start_node = node
        elif in_degrees[node] - out_degrees[node] == 1:
            if end_node:
                print("Multiple end nodes found. Graph is not Eulerian.")
                return None, G
            end_node = node
    
    if not start_node and not end_node:
        try:
            path = list(nx.eulerian_circuit(G))
            print("Eulerian circuit found.")
            return path, G
        except nx.NetworkXError:
            print("No Eulerian Path or Circuit found.")
            return None, G
    
    if (start_node and not end_node) or (end_node and not start_node):
        print("Graph is not Eulerian. Mismatched start and end nodes.")
        return None, G
    
    G.add_edge(end_node, start_node)
    
    try:
        path = list(nx.eulerian_circuit(G, source=start_node))
        path = [edge for edge in path if edge != (end_node, start_node)]
        print("Eulerian path found.")
        return path, G
    except nx.NetworkXError:
        print("No Eulerian Path found even after balancing. Graph might be disconnected.")
        return None, G

def reconstruct_sequence(path):
    return path[0][0] + ''.join(edge[1][-1] for edge in path)

def assemble_genome(reads, k):
    kmers = create_kmers(reads, k)
    de_bruijn_graph = build_de_bruijn_graph(kmers)
    eulerian_path, G = find_eulerian_path(de_bruijn_graph)
    
    if eulerian_path:
        reconstructed_sequence = reconstruct_sequence(eulerian_path)
        return reconstructed_sequence, G
    else:
        return None, G

# Main execution
if __name__ == "__main__":
    # Generate a random DNA sequence
    original_sequence = generate_random_dna(100)
    print("Original Sequence:", original_sequence)
    
    # Simulate reads
    read_length = 50
    num_reads = 100
    k = 31  # k-mer size
    
    reads = simulate_reads(original_sequence, read_length, num_reads)
    print(f"\nGenerated {num_reads} reads of length {read_length}")
    
    # Assemble the genome
    reconstructed_sequence, G = assemble_genome(reads, k)
    
    if reconstructed_sequence:
        print("\nReconstructed Sequence:", reconstructed_sequence)
        print("\nReconstruction success:", original_sequence in reconstructed_sequence)
        
        # Additional information
        print("\nGraph Information:")
        print(f"Nodes: {G.nodes()}")
        print(f"Edges: {G.edges()}")
        print("In-degrees:", dict(G.in_degree()))
        print("Out-degrees:", dict(G.out_degree()))
    else:
        print("\nFailed to reconstruct the sequence")

Original Sequence: ATCTTTAACCCAACGTCAGCATGGTGATGGCCCGTGCTCCCTTGGTATTACAGCCTACTGGGAGAAATTCCAGGACGGAGAAGGACACTGCAAAGATGGT

Generated 100 reads of length 50
Eulerian path found.

Reconstructed Sequence: ATCTTTAACCCAACGTCAGCATGGTGATGGCCCGTGCTCCCTTGGTATTACAGCCTACTGGGAGAAATTCCAGGACGGAGAAGGACACTGCAAAGATGGT

Reconstruction success: True

Graph Information:
Nodes: ['TTACAGCCTACTGGGAGAAATTCCAGGACG', 'TACAGCCTACTGGGAGAAATTCCAGGACGG', 'ACAGCCTACTGGGAGAAATTCCAGGACGGA', 'CAGCCTACTGGGAGAAATTCCAGGACGGAG', 'AGCCTACTGGGAGAAATTCCAGGACGGAGA', 'GCCTACTGGGAGAAATTCCAGGACGGAGAA', 'CCTACTGGGAGAAATTCCAGGACGGAGAAG', 'CTACTGGGAGAAATTCCAGGACGGAGAAGG', 'TACTGGGAGAAATTCCAGGACGGAGAAGGA', 'ACTGGGAGAAATTCCAGGACGGAGAAGGAC', 'CTGGGAGAAATTCCAGGACGGAGAAGGACA', 'TGGGAGAAATTCCAGGACGGAGAAGGACAC', 'GGGAGAAATTCCAGGACGGAGAAGGACACT', 'GGAGAAATTCCAGGACGGAGAAGGACACTG', 'GAGAAATTCCAGGACGGAGAAGGACACTGC', 'AGAAATTCCAGGACGGAGAAGGACACTGCA', 'GAAATTCCAGGACGGAGAAGGACACTGCAA', 'AAATTCCAGGACGGAGAAGGACACTGCAAA', 'AATTCCAGGACGGAGAAGGACACTGCAA